# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


In [3]:
len(pd_data.columns)

5

# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [4]:
## 각각의 줄이 무슨 의미인지 주석 구체적으로 달아주세요~~!!!
# 4개가 나눠질 때 두개 두개로 나눠지면 어떻게 게싼해야 할지 생각해보세요.
def get_gini(df, label):
    # 여기서 df는 df['age']에서 youth인 값들처럼 알고싶은 feature의 class는 이미 나눠져 있음
    '''
        이 부분을 채워주세요
        feature에 대한 각각 Gini 계수 구하기
    '''
    
    # 한 label에 대한 gini 계수 구하기
    class_list = df[label].unique()
    will_subtract = 0
#     print(class_list)
    for one_class in class_list: #label의 종류들만큼 값 더해서 나중에 1에 뺴주면 됨
#         print((df['class_buys_computer']==one_class).sum()/len(df))
        will_subtract += ((df['class_buys_computer']==one_class).sum()/len(df))**2
    
    gini = 1-will_subtract
    
    return gini    

In [8]:
pd_data['class_buys_computer'].unique()

array(['no', 'yes'], dtype=object)

In [9]:
(pd_data['class_buys_computer']=='yes')

0     False
1     False
2      True
3      True
4      True
5     False
6      True
7     False
8      True
9      True
10     True
11     True
12     True
13    False
Name: class_buys_computer, dtype: bool

In [10]:
df = pd_data.loc[(pd_data['age']=='senior')|(pd_data['age']=='middle_aged')]

In [11]:
# 더 편한 방법 없을까?-> isin
df = pd_data.loc[(pd_data['age'].isin(['senior','middle_aged']))]

In [24]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [13]:
# 조합으로 선택된 값을 빼면 나오는 요소를 다른 class로 잡고 계산하기
def get_rest_elem(selected_elem, all_elem):
    rest_elem = []
    for one_elem in all_elem:
        if one_elem not in selected_elem:
            rest_elem.append(one_elem)
    return rest_elem
    

In [14]:
get_rest_elem([1,2], [1,2,3])

[3]

In [15]:
import itertools
# 각각의 조합 찾기
list(itertools.combinations(['a','b','c','d'],2))

[('a', 'b'), ('a', 'c'), ('a', 'd'), ('b', 'c'), ('b', 'd'), ('c', 'd')]

In [16]:
# 모든 조합을 보려면 nC1+nC2+...+nCn-1 을 더한 값 마늠 보면 됨 총 2^n-2개만큼 나옴
# 하지만, 모든 split에서 최소값 구하면 되니까 굳이 어떤 쌍으로 split 되는지는 볼 필요가 없음
# column별로 묶음 조합 만들기(5개일때 2개가 만들어지면 3개는 자동으로 만들어지므로 반의 몫만큼만 돌리면 된다)
def get_binary_split(df, attribute):
    split_list = []
    for i in range(1,len(df[attribute].unique())):
        selected_elem = list(itertools.combinations(df[attribute].unique(),i))
        # 원래는 split이니까 하나를 뽑으면 하나는 자동으로 나오니까 따로 얻으로고 했음 -> 나중에 필요하면 함수 구분
        # rest_elem = get_rest_elem(selected_elem, list(df[attribute].unique()))
        split_list.append(selected_elem)
#     print(split_list)
    # 답은 flatten 된거니까 flatten
    split_list_flatten = [one_comb for one_split in split_list for one_comb in one_split]
    return split_list_flatten


In [17]:
get_binary_split(pd_data,"age")

[('youth',),
 ('middle_aged',),
 ('senior',),
 ('youth', 'middle_aged'),
 ('youth', 'senior'),
 ('middle_aged', 'senior')]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [18]:
df['age'].unique()

array(['middle_aged', 'senior'], dtype=object)

In [19]:
get_rest_elem(('youth',),pd_data['age'].unique())

['middle_aged', 'senior']

In [20]:
'_'.join(list(('youth', 'middle_aged')))

'youth_middle_aged'

In [76]:
def get_attribute_gini_index(df, attribute, label):
    '''
        이 부분을 채워주세요
    '''
    
    split_list = get_binary_split(df,attribute)
    gini_dict = {}
    for one_split in split_list:
        result = 0
        # 해당 split에서 있는 category만 필터링하기
        filtered_df = df.loc[(df[attribute].isin(one_split))]
        filetred_df_ratio = len(filtered_df)/len(df)
#         print(filtered_df['age'].unique())
#         print('filetred_df_ratio',filetred_df_ratio)
        result += filetred_df_ratio*get_gini(filtered_df,label)
        
        # 해당 split을 선택하면 나오는 나머지 split도 계산하기
        pair_split = get_rest_elem(one_split,df[attribute].unique())
        pair_filtered_df = df.loc[(df[attribute].isin(pair_split))]
        pair_filetred_df_ratio = len(pair_filtered_df)/len(df)
        result += pair_filetred_df_ratio*get_gini(pair_filtered_df,label)
        
        # 이름 _로 연결하기
        key_name = '_'.join(list(one_split))
        gini_dict[key_name] = result

    return gini_dict

In [69]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.45714285714285713,
 'youth_middle_aged': 0.45714285714285713,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [70]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items(),key=lambda x:x[1])

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

#### 문제1

In [71]:
##문제1 답안
get_binary_split(pd_data,"income")


[('high',),
 ('medium',),
 ('low',),
 ('high', 'medium'),
 ('high', 'low'),
 ('medium', 'low')]

#### 문제2

In [72]:
##문제2 답안


In [73]:
pd_data.columns

Index(['age', 'income', 'student', 'credit_rating', 'class_buys_computer'], dtype='object')

In [74]:
for col in pd_data.columns:
    if col != 'class_buys_computer':
        print(col)
        print(get_attribute_gini_index(pd_data, col, "class_buys_computer").items())
        print(min(get_attribute_gini_index(pd_data, col, "class_buys_computer").items()))

age
dict_items([('youth', 0.3936507936507937), ('middle_aged', 0.35714285714285715), ('senior', 0.45714285714285713), ('youth_middle_aged', 0.45714285714285713), ('youth_senior', 0.35714285714285715), ('middle_aged_senior', 0.3936507936507937)])
('middle_aged', 0.35714285714285715)
income
dict_items([('high', 0.14285714285714285), ('medium', 0.19047619047619047), ('low', 0.10714285714285714), ('high_medium', 0.34285714285714286), ('high_low', 0.26785714285714285), ('medium_low', 0.30000000000000004)])
('high', 0.14285714285714285)
student
dict_items([('no', 0.24489795918367352), ('yes', 0.12244897959183676)])
('no', 0.24489795918367352)
credit_rating
dict_items([('fair', 0.21428571428571427), ('excellent', 0.21428571428571427)])
('excellent', 0.21428571428571427)


# 최소값 계산이 이상하게 된다+ 코드실수
dict_items([('no', 0.24489795918367352), ('yes', 0.12244897959183676)])

('no', 0.24489795918367352)

In [78]:
# key를 명확하게 지정해준다 ,key=lambda x:x[1]
for col in pd_data.columns:
    if col != 'class_buys_computer':
        print(col)
        print(get_attribute_gini_index(pd_data, col, "class_buys_computer").items())
        print(min(get_attribute_gini_index(pd_data, col, "class_buys_computer").items(),key=lambda x:x[1]))

age
dict_items([('youth', 0.3936507936507937), ('middle_aged', 0.35714285714285715), ('senior', 0.45714285714285713), ('youth_middle_aged', 0.45714285714285713), ('youth_senior', 0.35714285714285715), ('middle_aged_senior', 0.3936507936507937)])
('middle_aged', 0.35714285714285715)
income
dict_items([('high', 0.4428571428571429), ('medium', 0.4583333333333333), ('low', 0.45), ('high_medium', 0.45), ('high_low', 0.4583333333333333), ('medium_low', 0.4428571428571429)])
('high', 0.4428571428571429)
student
dict_items([('no', 0.3673469387755103), ('yes', 0.3673469387755103)])
('no', 0.3673469387755103)
credit_rating
dict_items([('fair', 0.42857142857142855), ('excellent', 0.42857142857142855)])
('fair', 0.42857142857142855)


### -> 값이 작을 수록 좋다. age가 가장 작고 지니 계수는 0.35714285714285715이다

#### 문제3

In [82]:
pd_data['age'].unique()

array(['youth', 'middle_aged', 'senior'], dtype=object)

In [85]:
# income에서 high / medium, low로 나눈다.
middle = pd_data.loc[pd_data['age']=='middle_aged']
middle_complement = pd_data.loc[pd_data['age']!='middle_aged']

In [86]:
middle

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [88]:
for col in high.columns:
    if col not in ['class_buys_computer','age']:
        print(col)
        print(get_attribute_gini_index(middle, col, "class_buys_computer"))
        print(min(get_attribute_gini_index(middle, col, "class_buys_computer").items(),key=lambda x:x[1]))

income
{'high': 0.0, 'low': 0.0, 'medium': 0.0, 'high_low': 0.0, 'high_medium': 0.0, 'low_medium': 0.0}
('high', 0.0)
student
{'no': 0.0, 'yes': 0.0}
('no', 0.0)
credit_rating
{'fair': 0.0, 'excellent': 0.0}
('fair', 0.0)


#### -> 여기서 나누는걸 끝내면 된다

In [89]:
middle_complement

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes
10,youth,medium,yes,excellent,yes
13,senior,medium,no,excellent,no


In [92]:
for col in high_complement.columns:
    if col not in ['class_buys_computer','age']:
        print(col)
        print(get_attribute_gini_index(middle_complement, col, "class_buys_computer"))
        print(min(get_attribute_gini_index(middle_complement, col, "class_buys_computer").items(),key=lambda x:x[1]))

income
{'high': 0.375, 'medium': 0.48, 'low': 0.4761904761904763, 'high_medium': 0.4761904761904763, 'high_low': 0.48, 'medium_low': 0.375}
('high', 0.375)
student
{'no': 0.31999999999999984, 'yes': 0.31999999999999984}
('no', 0.31999999999999984)
credit_rating
{'fair': 0.4166666666666667, 'excellent': 0.4166666666666667}
('fair', 0.4166666666666667)


#### -> high가 아닌 애들은 student로 나누면 좋다

# 과제2(앗ㅋㅋㅋ)

In [ ]:
# 전체 entropy
import math
whole_entropy = -9/14*math.log(9/14,2)-5/14*math.log(5/14,2)

In [ ]:
whole_entropy

In [ ]:
# 각 feature별 entropy
age_entropy = (5/14*(-3/5*math.log(3/5,2)-2/5*math.log(2/5,2))) + (4/14*(-4/4*math.log(4/4,2))) + 5/14*(-3/5*math.log(3/5,2)-2/5*math.log(2/5,2))

In [ ]:
age_entropy

In [ ]:
income_entropy = (4/14*(-2/4*math.log(2/4,2)-2/4*math.log(2/4,2))) + (6/14*(-4/6*math.log(4/6,2)-2/6*math.log(2/6,2))) + 4/14*(-3/4*math.log(3/4,2)-1/4*math.log(1/4,2))

In [ ]:
income_entropy

In [ ]:
student_entropy = (7/14*(-4/7*math.log(4/7,2)-3/7*math.log(4/7,2))) + (7/14*(-6/7*math.log(6/7,2)-1/7*math.log(1/7,2)))

In [ ]:
student_entropy

In [ ]:
credit_entropy = (8/14*(-2/8*math.log(2/8,2)-6/8*math.log(6/8,2))) + (6/14*(-3/6*math.log(3/6,2)-3/6*math.log(3/6,2)))

In [ ]:
credit_entropy

In [ ]:
[whole_entropy-entropy for entropy in [age_entropy, income_entropy, student_entropy, credit_entropy]]